# Comprehensive Data Processing and Optimization for Brand Matching

This script adeptly handles both data processing and the optimization of brand pairings using advanced algorithmic techniques. It starts by processing Instagram post and follower data loaded from two separate Excel files. Key operations include formatting date columns for consistent datetime objects, pruning unnecessary columns to streamline the dataset, and converting various columns to categorical types to better represent the underlying data. Additionally, it assigns categorical labels to numeric ranges, enhancing the interpretability of data points such as comments and follower counts.

The script then applies the Hungarian algorithm, a robust combinatorial optimization method, to determine optimal brand pairings. This method effectively minimizes the total dissimilarity across matched pairs based on a calculated distance matrix from one-hot encoded features, ensuring each brand is uniquely paired with the most similar counterpart. This dual approach of meticulous data processing followed by strategic optimization makes this script particularly valuable for generating actionable insights in brand management and market analysis.

## Step 1 - Advanced Data Processing and Transformation Techniques for Instagram Analytics

1. **Data Loading**:
   - **Description**: Loads Instagram posts and follower data from Excel files.
   - **Files**:
     - `insta_posts_1.xlsx` contains the Instagram posts data.
     - `insta_followers_2.xlsx` contains the Instagram follower data.

2. **Data Preparation**:
   - **Date Conversion**:
     - Converts the 'Date' columns in both dataframes to Python datetime objects to facilitate date manipulations.
   - **Column Removal**:
     - Removes the 'Likes' column from the `insta_posts` dataframe as it may not be needed for further analysis.

3. **Function Definition**:
   - **find_closest_date**:
     - A function defined to find the closest date in `insta_followers` for each date in `insta_posts` and retrieve the corresponding follower count.
     - Utilizes the minimum time difference between dates to determine the closest match.

4. **Applying Functions**:
   - **Description**: Applies the `find_closest_date` function to each post's date to append the corresponding follower count from the closest date in the `insta_followers` dataframe.

5. **Data Transformation**:
   - **Categorical Conversion**:
     - Converts the 'Type' and 'Month' columns to categorical data types, with 'Month' extracted from the 'Date' column.
   - **Binning Numeric Columns**:
     - Bins the 'Comments' and 'Follower' columns into predefined categorical labels based on specified numeric ranges.

6. **Encoding Types**:
   - **Description**: Converts the 'Type' column to a numerical representation using factorization, which assigns a unique integer to each type.
   - **Output**: A new column `Encoded_Type` is added to the `insta_posts` dataframe.

7. **File Export**:
   - **Description**: Exports the processed `insta_posts` dataframe to an Excel file named `encoded_insta_posts2.xlsx` without the index.

8. **Verification**:
   - **Description**: Displays the head of the updated `insta_posts` dataframe to verify changes.
   - **Additional Information**: Optionally prints a mapping of numerical encodings to post types to clarify the transformations.

In [1]:
from datetime import datetime
import pandas as pd

insta_posts = pd.read_excel("C:/Users/wonny/Downloads/insta_posts_1.xlsx")
insta_followers = pd.read_excel("C:/Users/wonny/Downloads/insta_followers_2.xlsx")

# Convert 'Date' columns to datetime
insta_posts["Date"] = pd.to_datetime(insta_posts["Date"], format="%Y-%m-%d %I:%M %p")
insta_followers["Date"] = pd.to_datetime(insta_followers["Date"], format="%d %B %Y")

# Drop the 'Likes' column from insta_posts
insta_posts.drop("Likes", axis=1, inplace=True)


# Create a function to find the closest date in insta_followers for each date in insta_posts
def find_closest_date(post_date, followers_df):
    # Calculate the absolute difference between the post date and all follower dates
    followers_df["time_diff"] = (followers_df["Date"] - post_date).abs()
    # Find the row with the smallest time difference
    closest_row = followers_df.loc[followers_df["time_diff"].idxmin()]
    # Return the follower count from the closest date
    return closest_row["Follower"]


# Apply the function to each post date
insta_posts["Follower"] = insta_posts["Date"].apply(
    lambda date: find_closest_date(date, insta_followers.copy())
)

# Display the updated insta_posts dataframe
insta_posts.head()

,Brands,Type,Date,Engagement,Comments,Follower
0,adidasoriginals,reel,2023-09-25 08:00:00,1048657,36385,1063809
1,adidasoriginals,reel,2023-09-28 09:08:00,397481,751,34408717
2,adidasoriginals,CAROUSEL_ALBUM,2023-05-04 04:00:00,322175,4663,34315247
3,adidasoriginals,CAROUSEL_ALBUM,2024-04-23 11:00:00,283118,674,34488386
4,adidasoriginals,reel,2023-09-28 04:12:00,219199,9594,34408717


In [2]:
# Convert 'Type' column to a categorical variable
insta_posts["Type"] = insta_posts["Type"].astype("category")

# Convert 'Date' column to month and then to a categorical variable
insta_posts["Month"] = insta_posts["Date"].dt.month.astype("category")

# Define bins for the 'Comments' and labels
comments_bins = [0, 50, 200, 1000, 5000, 10000, float("inf")]
comments_labels = ["0-50", "50-200", "200-1K", "1K-5K", "5K-10K", "10K+"]
insta_posts["Comments"] = pd.cut(
    insta_posts["Comments"], bins=comments_bins, labels=comments_labels
)

# Assuming the 'Follower' column is numerical and needs to be binned similarly
# Adjust the bins and labels according to the expected follower count ranges
follower_bins = [0, 1000, 5000, 20000, 100000, 500000, float("inf")]
follower_labels = ["0-1K", "1-5K", "5-20K", "20-100K", "100-500K", "500K+"]
insta_posts["Follower"] = pd.cut(
    insta_posts["Follower"], bins=follower_bins, labels=follower_labels
)

# Now that all the specified columns are converted to categorical types, you can use the dataframe as needed
# Display the updated dataframe to verify the changes
insta_posts.head()

,Brands,Type,Date,Engagement,Comments,Follower,Month
0,adidasoriginals,reel,2023-09-25 08:00:00,1048657,10K+,500K+,9
1,adidasoriginals,reel,2023-09-28 09:08:00,397481,200-1K,500K+,9
2,adidasoriginals,CAROUSEL_ALBUM,2023-05-04 04:00:00,322175,1K-5K,500K+,5
3,adidasoriginals,CAROUSEL_ALBUM,2024-04-23 11:00:00,283118,200-1K,500K+,4
4,adidasoriginals,reel,2023-09-28 04:12:00,219199,5K-10K,500K+,9


In [3]:
# Convert 'Type' column to numerical representation
encoded_types, type_mapping = pd.factorize(insta_posts["Type"])

# Create a new column for the encoded types
insta_posts["Encoded_Type"] = encoded_types

# Optionally, print the mapping to see which number corresponds to which type
print("Type Mapping:", type_mapping)

insta_posts

Type Mapping: CategoricalIndex(['reel', 'CAROUSEL_ALBUM', 'IMAGE', 'VIDEO', 'IGTV'], categories=['CAROUSEL_ALBUM', 'IGTV', 'IMAGE', 'VIDEO', 'reel'], ordered=False, dtype='category')


,Brands,Type,Date,Engagement,Comments,Follower,Month,Encoded_Type
0,adidasoriginals,reel,2023-09-25 08:00:00,1048657,10K+,500K+,9,0
1,adidasoriginals,reel,2023-09-28 09:08:00,397481,200-1K,500K+,9,0
2,adidasoriginals,CAROUSEL_ALBUM,2023-05-04 04:00:00,322175,1K-5K,500K+,5,1
3,adidasoriginals,CAROUSEL_ALBUM,2024-04-23 11:00:00,283118,200-1K,500K+,4,1
4,adidasoriginals,reel,2023-09-28 04:12:00,219199,5K-10K,500K+,9,0
...,...,...,...,...,...,...,...,...
10914,veja,CAROUSEL_ALBUM,2023-09-05 12:36:00,479,0-50,500K+,9,1
10915,veja,IMAGE,2024-02-12 09:08:00,460,0-50,500K+,2,2
10916,veja,CAROUSEL_ALBUM,2023-08-22 11:30:00,440,0-50,500K+,8,1
10917,veja,CAROUSEL_ALBUM,2023-07-04 11:30:00,430,0-50,500K+,7,1


In [4]:
insta_posts.to_excel("encoded_insta_posts2.xlsx", index=False)

## Step 2 - Matching Approach

This script utilizes the Hungarian algorithm, a combinatorial optimization method ideally suited for solving assignment problems like optimal brand pairing. By converting all categorical variables into a one-hot encoded format, the script ensures that each brand is represented as a multidimensional point, facilitating the computation of a Euclidean distance matrix that captures dissimilarities between brands. The Hungarian algorithm then optimally matches each brand to another based on this distance matrix, minimizing total dissimilarity across all pairings and ensuring each brand is uniquely matched.

The effectiveness of this approach lies in its ability to provide objective, data-driven insights into brand similarities, which are crucial for competitive analysis and strategic decision-making. By guaranteeing optimal and unique brand pairings, the algorithm supports clear and logical market analysis, eliminating subjective biases and enabling precise, strategic insights. This method not only ensures that each brand is matched exactly once, maintaining clarity and efficiency in data handling, but it also supports businesses in making informed decisions based on robust mathematical foundations and objective criteria.


The provided code performs brand matching using several well-defined steps, each encapsulated within a function:

1. **Loading Data**:
   - **Function**: `load_data`
   - **Description**: Reads data from an Excel file into a pandas DataFrame.
   - **Input**: File path.
   - **Output**: DataFrame.

2. **Preprocessing Data**:
   - **Function**: `preprocess_data`
   - **Description**: Converts categorical columns to one-hot encoded formats and aggregates data by 'Brands'.
   - **Input**: DataFrame.
   - **Output**: Aggregated DataFrame with one-hot encoded values.

3. **Calculating Distance Matrix**:
   - **Function**: `calculate_distance_matrix`
   - **Description**: Computes a pairwise Euclidean distance matrix from one-hot encoded features.
   - **Input**: DataFrame with encoded features.
   - **Output**: Pairwise distance matrix, with self-matching prevented by setting diagonal values to infinity.

4. **Finding Optimal Pairs**:
   - **Function**: `find_optimal_pairs`
   - **Description**: Applies the linear sum assignment algorithm to find optimal pairings based on the distance matrix.
   - **Input**: Distance matrix and DataFrame with brands.
   - **Output**: List of tuples representing paired brands.

5. **Printing Pairs**:
   - The final step is to print each pair, showing the results of the matching process.

In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.optimize import linear_sum_assignment


def load_data(file_path):
    """
    Load data from an Excel file into a pandas DataFrame.

    Args:
        file_path (str): The path to the Excel file.

    Returns:
        DataFrame: Loaded data as pandas DataFrame.
    """
    return pd.read_excel(file_path)


def preprocess_data(data):
    """
    Convert categorical columns to one-hot encoded format and aggregate data by 'Brands'.

    Args:
        data (DataFrame): The DataFrame to preprocess.

    Returns:
        DataFrame: The preprocessed and aggregated data.
    """
    categorical_columns = ["Encoded_Type", "Comments", "Follower", "Month"]
    data_encoded = pd.get_dummies(data, columns=categorical_columns)
    return data_encoded.groupby("Brands").mean(numeric_only=True).reset_index()


def calculate_distance_matrix(data):
    """
    Calculate the pairwise Euclidean distance matrix for the encoded features.

    Args:
        data (DataFrame): The DataFrame with one-hot encoded categorical features.

    Returns:
        ndarray: The pairwise distance matrix.
    """
    features = data.columns[3:]  # Exclude 'Brands' and 'Engagement'
    distance_matrix = pairwise_distances(data[features], metric="euclidean")
    np.fill_diagonal(distance_matrix, np.inf)  # Prevent self-matching
    return distance_matrix


def find_optimal_pairs(distance_matrix, data):
    """
    Use the linear sum assignment method to find optimal pairings based on the distance matrix.

    Args:
        distance_matrix (ndarray): The matrix of distances between each pair of items.
        data (DataFrame): DataFrame containing the brands information.

    Returns:
        list of tuples: List containing pairs of matched brands.
    """
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    paired_indices = set()
    pairs = []
    for row, col in zip(row_ind, col_ind):
        if row < col and row not in paired_indices and col not in paired_indices:
            pairs.append((data["Brands"].iloc[row], data["Brands"].iloc[col]))
            paired_indices.update([row, col])

    # Check if all brands are paired
    if len(pairs) != 10:
        unpaired_brands = list(set(range(len(data))) - paired_indices)
        if len(unpaired_brands) == 2:
            pairs.append(
                (
                    data["Brands"].iloc[unpaired_brands[0]],
                    data["Brands"].iloc[unpaired_brands[1]],
                )
            )
    return pairs


# Main execution
data_path = "C:/Users/wonny/Downloads/encoded_insta_posts2.xlsx"
data = load_data(data_path)
processed_data = preprocess_data(data)
distance_matrix = calculate_distance_matrix(processed_data)
pairs = find_optimal_pairs(distance_matrix, processed_data)

# Print the pairs
for pair in pairs:
    print(f"{pair[0]} & {pair[1]}")

adidasoriginals & vinted
allbirds & urbanic
asos & stockx
everlane & lenskart
ganni & mejuri
goat & veja
lunya & rixo
pomelofashion & printful
prettylittlething & skims
fashionphile & primark
